# Import Required Packages

In [ ]:
import requests
import pandas as pd
import urllib
import csv
import time
from os import path
import datetime

In [ ]:
githubAccessToken = ''
fileName = 'issues.csv'

# Define utility functions

In [ ]:
def ensureRequestCount(r):
    remaining = int(r.headers['X-RateLimit-Remaining'])
    print("Remaining limit: " + str(remaining))
    if (remaining == 0):
        reset_time = datetime.datetime.fromtimestamp(int(r.headers['X-RateLimit-Reset']))
        seconds_until_reset = (reset_time - datetime.datetime.now()).total_seconds() + 10
        print("Limit Exceed, Going to wait mode for " + str(seconds_until_reset))
        time.sleep(seconds_until_reset)
        
def getRequest(url):
    if (githubAccessToken!=''):
        headers = {'Authorization': 'token '+ githubAccessToken}
    else:
        headers = ''
    response = requests.get(url,headers=headers)
    ensureRequestCount(response)
    return response

def getUrlForPage(url,page):
    return url + "?per_page=100&page="+str(page)

# Extract issues from the repositories
Here we extract the issues from the repositories collected as a result of running the 'Extract Repos' notebook.

In [ ]:
df = pd.read_csv('repos.csv')
totalRepos = len(df.index)
fetchedCount = 0
lastFetched = '' # Incase of unexplained failure, copy the last fetched URL here
lastFetchedPage = 1 # Incase of failure, copy the last fetched page number here
shouldResume = (lastFetched == None or lastFetched == '')
for repositoryRow in df.itertuples():
    fetchedCount += 1
    issue_url = repositoryRow[4]
    issue_url = issue_url.replace("{/number}","")
    currentPage = 1
    
    if (lastFetched != None and shouldResume == False):
        if (issue_url == lastFetched):
            shouldResume = True
            currentPage = lastFetchedPage + 1
        else:
            #print ("Skipped "+issue_url)
            continue
            
    
    print("------------- Repository " + str(fetchedCount) + " out of " + str(totalRepos) + "-------------")

    with open(fileName, 'a', newline='', encoding="utf8") as csvfile:
        while(True):
            paged_issue_url = getUrlForPage(issue_url,currentPage)
            while(True):
                try:
                    r = getRequest(paged_issue_url)
                    break
                except:
                    time.sleep(60)
                    continue
                    
            print(paged_issue_url)
            rObject = r.json()
            if (len(rObject) == 0):
                break
            try:
                if (rObject[0]['title'] != None):
                    pass
            except:
                print("HTTP Error at " +str(paged_issue_url)+ ". Skipping ...")
                break


            for issue in rObject:
                isNewFile = False
                if path.exists(fileName)==False:
                    isNewFile = True
                writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                if (isNewFile):
                    writer.writerow(['title','labels','state','body','issue_url','repository_url'])
                    isNewFile = False
                labels = []
                for label in issue['labels']:
                    labels.append({
                        'name:': label['name'],
                        'description:': label['description'],
                        'color': label['color']
                    })
                writer.writerow([issue['title'], labels, issue['state'], issue['body'], issue['url'], issue['repository_url']])
            currentPage+=1